### Assignment: Real-Time Market Sentiment Analyzer Using LangChain Chains

#### Objective
Build a LangChain-powered pipeline (Chain) that:
1. Accepts a company name as input.
2. Extracts or generates its stock code.
3. Uses News search tools in LangChain to fetch news about the company.
4. Sends the news to an LLM (Google Gemini-2.0-flash) to generate a structured 
sentiment profile.
5. Outputs the result as a JSON object.
6. Uses mlflow for tracing, prompt debugging, and monitoring.

### Installing the packages

In [2]:
!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
!python -m pip install -U langchain-google-genai --quiet
%pip install --upgrade --quiet  yfinance
%pip install langchain wikipedia-api --quiet
%pip install wikipedia --quiet
%pip install mlflow --quiet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

In [1]:
from google import genai
from google.genai.types import HttpOptions
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from pydantic import BaseModel, Field
from typing import Dict, Any




from langchain_core.output_parsers import StrOutputParser,JsonOutputParser


parser = StrOutputParser()
parser_1=JsonOutputParser()

###Intialising the client
client = genai.Client(http_options=HttpOptions(api_version='v1'))
from pydantic import BaseModel, Field
import pprint

USER_AGENT environment variable not set, consider setting it to identify your requests.


### assigning the model

In [2]:
model_name='gemini-2.0-flash'

### Initialising the model

In [3]:
model = init_chat_model(model_name, model_provider="google_genai")

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

### Setting the ML flow Tracking URI/

In [ ]:
import mlflow
mlflow.set_tracking_uri("enter the ML Flow tracking url here")


In [ ]:
mlflow.set_experiment("SG_Sentiment_Analysis")

<Experiment: artifact_location='/home/zadmin/Desktop/test/GAAI-B5-GCP/ML_FLOW_TRACKING_URI/932135586498962336', creation_time=1758278544166, experiment_id='932135586498962336', last_update_time=1758278544166, lifecycle_stage='active', name='SG_Sentiment_Analysis_1', tags={}>

#### Entering the company name

In [6]:
company_name=input("Enter the company name:")
print(company_name)

Tesla Inc


### Stock Ticker Library

In [7]:

stock_tickers = {
    "Apple Inc": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Amazon.com Inc": "AMZN",
    "Alphabet Inc": "GOOGL",
    "NVIDIA Corporation": "NVDA",
    "Meta Platforms Inc": "META",
    "Broadcom Inc": "AVGO",
    "Tesla Inc": "TSLA",
    "Taiwan Semiconductor": "TSM",
    "Berkshire Hathaway Inc": "BRK.B",
    "JPMorgan Chase & Co": "JPM",
    "Walmart Inc": "WMT",
    "Oracle Corporation": "ORCL",
    "Eli Lilly and Company": "LLY",
    "Visa Inc": "V",
    "Mastercard Inc": "MA",
    "Netflix Inc": "NFLX",
    "Exxon Mobil Corporation": "XOM",
    "Costco Wholesale Corporation": "COST",
    "Johnson & Johnson": "JNJ",
    "The Home Depot Inc": "HD",
    "Palantir Technologies Inc": "PLTR",
    "AbbVie Inc": "ABBV",
    "Bank of America Corporation": "BAC",
    "Procter & Gamble": "PG",
    "UnitedHealth Group": "UNH",
    "Chevron Corporation": "CVX",
    "ASML Holding N.V.": "ASML",
    "SAP SE": "SAP",
    "General Electric Company": "GE",
    "The Coca-Cola Company": "KO",
    "T-Mobile US Inc": "TMUS",
    "Cisco Systems Inc": "CSCO",
    "Wells Fargo & Company": "WFC",
    "Philip Morris International": "PM",
    "Advanced Micro Devices Inc": "AMD",
    "Toyota Motor Corporation": "TM",
    "Morgan Stanley": "MS",
    "AstraZeneca PLC": "AZN",
    "The Goldman Sachs Group Inc": "GS"
}


### Extract the stock ticker from user entered comapny name

In [8]:
### A function to extract the stock ticker or symbol
@mlflow.trace
def get_stock_ticker(company_name):
    lookup = stock_tickers
    return lookup.get(company_name, "UNKNOWN")
stock_code = get_stock_ticker(company_name)
print(stock_code)


TSLA


Trace(trace_id=tr-aab4fcba427646c4035cd5e716bf9344)

In [ ]:




# tool = YahooFinanceNewsTool()
# tool.invoke("AAPL")

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 3. Uses News search tools in LangChain to fetch news about the company

In [9]:


class StockNewsTool(BaseModel):
    """Inputs to the Stock News tool."""

    query: str = Field(
        description="query to look up in Yahoo Finance News, should be stock symbols"
    )
@mlflow.trace
def company_news_func(stock_code_1:str):
    Company_news_tool = YahooFinanceNewsTool(

    name="Stock-News Tool",
    description="look up the latest stock news of a company in Yahoo finance news using the extracted stock symbol.Extract a concise list of recent headlines,article summaries as well as historical stock data if available.",
    args_schema=StockNewsTool,
    metadata={"use_case": "financial_analysis", "source": "YahooFinance"},
   
    return_direct=True,
    
    top_k= 5,
    verbose=True

)
    return Company_news_tool.run(stock_code_1)




# print(tool.run("AAPL"))
response=company_news_func(stock_code)

pprint.pprint(response)

Tesla (TSLA) Opens Cybertruck Orders in UAE
Tesla, Inc. (NASDAQ:TSLA) is one of the 10 Best NASDAQ Stocks to Buy For Long Term. On September 16, Tesla, Inc. (NASDAQ:TSLA) announced on X that it has officially opened Cybertruck orders in the United Arab Emirates (UAE), allowing reservation holders in the country to order. This marks the beginning of Tesla, Inc.’s (NASDAQ:TSLA) launch […]

Jim Cramer Notes Elon Musk’s Significant Purchase of Tesla Stock
Tesla, Inc. (NASDAQ:TSLA) is one of the stocks Jim Cramer weighed in on. Cramer discussed the company CEO’s purchase of $1 billion worth of stock recently, as he stated: “When we came up with the term FANG, which stood for, at that point, Meta, Amazon, Netflix, and then Alphabet a dozen years ago, all I […]

Tesla Stock (TSLA) Backed by Morgan Stanley on Physical AI Opportunity
Tesla, Inc. (NASDAQ:TSLA) is one of the AI Stocks Making Headlines This Week. On September 16, Morgan Stanley reiterated the stock as “Overweight” and said it’s st

Trace(trace_id=tr-13f411b14cea775414886805dc4578d4)

### Step 4: Analyze Sentiment with Google Gemini-2.0-flash
• Pass the news summaries to the Gemini model via LangChain with a prompt that asks 
the LLM to:
o Classify sentiment.
o Extract named entities: people, places, other companies.
o Provide a structured JSON with the following fields:

In [10]:
from langchain.chains import LLMChain

In [11]:
mlflow.gemini.autolog()

### Registering the prompt via MLflow

In [12]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """
You are a financial analyst. Given the following news about {company} {ticker}, analyze the sentiment and extract structured insights.

News:
{news}

Return a dictionary with the following fields:
{{
 "company_name": "{company}",
 "stock_code": "{ticker}",
 "newsdesc": "...",
 "sentiment": "Positive/Negative/Neutral",
 "people_names": [],
 "places_names": [],
 "other_companies_referred": [],
 "related_industries": [],
 "market_implications": "",
 "confidence_score": 0.0
}}
"""


# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="SG-SentimentAnalysis-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Soumee Ghosh",
        "task": "Sentiment analysis",
        "language": "en",
        'llm': 'gemini-2.0-flash'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/09/19 14:47:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: SG-SentimentAnalysis-prompt, version 6


Created prompt 'SG-SentimentAnalysis-prompt' (version 6)


### Loading the prompt from MLflow

In [13]:
sentiment_prompt = mlflow.genai.load_prompt("prompts:/SG-SentimentAnalysis-prompt/4")
print(sentiment_prompt.template)


You are a financial analyst. Given the following news about {company} {ticker}, analyze the sentiment and extract structured insights.

News:
{news}

Return a dictionary with the following fields:
{{
 "company_name": "{company}",
 "stock_code": "{ticker}",
 "newsdesc": "...",
 "sentiment": "Positive/Negative/Neutral",
 "people_names": [],
 "places_names": [],
 "other_companies_referred": [],
 "related_industries": [],
 "market_implications": "",
 "confidence_score": 0.0
}}



### Sends the news to an LLM (Google Gemini-2.0-flash) to generate a structured sentiment profile.

### What are Langchain Chains?


Chains are easily reusable components linked together.

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc., and provide a simple interface to this sequence.

The Chain interface makes it easy to create apps that are:

Stateful: add Memory to any Chain to give it state,

Observable: pass Callbacks to a Chain to execute additional functionality, like logging, outside the main sequence of component calls,

Composable: combine Chains with other components, including other Chains.

In [14]:

@mlflow.trace
def run_sentiment_analysis(model, sentiment_prompt, company_name, stock_code, news_text):
    """
    Run sentiment analysis on news related to a company using an LLM chain.

    Args:
        model: The language model instance (e.g., from OpenAI, HuggingFace, etc.).
        sentiment_prompt: A PromptTemplate object or template string for sentiment analysis.
        company_name (str): The name of the company.
        stock_code (str): The ticker symbol of the company.
        news_text (str): The news article or snippet for analysis.

    Returns:
        str: The model's response (e.g., sentiment analysis result).
    """
    
    # Create the PromptTemplate object if it's not already
    if isinstance(sentiment_prompt, str):
        prompt_1 = PromptTemplate(
            input_variables=["company", "ticker", "news"],
            template=sentiment_prompt
        )
    else:
        prompt_1 = sentiment_prompt

    # Create the LLM chain and run it
    chain = LLMChain(llm=model, prompt=prompt_1)
    response_final = chain.run(company=company_name, ticker=stock_code, news=news_text)
    response_final=parser_1.invoke(response_final)
    
    return response_final




In [15]:
final_response = run_sentiment_analysis(
    model=model,
    sentiment_prompt=sentiment_prompt.template,
    company_name=company_name,
    stock_code=stock_code,
    news_text=response
)

/tmp/ipykernel_4257/4016947636.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt_1)
/tmp/ipykernel_4257/4016947636.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_final = chain.run(company=company_name, ticker=stock_code, news=news_text)


Trace(trace_id=tr-dee88c8ce6784bdcb977cc336d101fc3)

In [16]:
final_response

{'company_name': 'Tesla Inc',
 'stock_code': 'TSLA',
 'newsdesc': 'Tesla opens Cybertruck orders in UAE; Jim Cramer notes Elon Musk’s significant stock purchase; Morgan Stanley backs Tesla stock on physical AI opportunity.',
 'sentiment': 'Positive',
 'people_names': ['Elon Musk', 'Jim Cramer'],
 'places_names': ['United Arab Emirates', 'UAE'],
 'other_companies_referred': ['Meta', 'Amazon', 'Netflix', 'Alphabet'],
 'related_industries': ['Automotive',
  'Electric Vehicles',
  'Artificial Intelligence'],
 'market_implications': "Positive indicators for Tesla, including expansion into new markets (UAE), CEO's confidence reflected in stock purchase, and analyst support based on AI potential. This suggests potential upward pressure on the stock price.",
 'confidence_score': 0.85}

### Bonus Ideas (Optional)
• Add entity linking for extracted people and companies (e.g., using Wikipedia or 
LinkedIn).

In [19]:
entity_name=final_response["people_names"]
entity_name=str(entity_name)
print(entity_name)
type(entity_name)

['Elon Musk', 'Jim Cramer']


str

In [20]:





class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: Any= Field(
        description="query to look up in Wikipedia, should be a list"
    )

@mlflow.trace
def wiki_tool_func(entity_name_1:str):
    wiki_tool = WikipediaQueryRun(
    name="wiki-tool",
    description="Look up the latest information about the entity_name specified in the wikipedia and return a concise 2 line response highlighting financial news related to the entity",
    args_schema=WikiInputs,
    api_wrapper=WikipediaAPIWrapper(top_k_results=2),

    
    
    return_direct=True,
    metadata={"use_case": "entity_analysis", "source": "Wikipedia"}
)
    return wiki_tool.run(entity_name)


wiki_tool_func(entity_name)

'Page: Elon Musk salute controversy\nSummary: On January 20, 2025, while speaking at a rally celebrating U.S. president Donald Trump\'s second inauguration, businessman and political figure Elon Musk twice made a salute interpreted by many as a Nazi or a fascist Roman salute.\nIt was widely condemned as an intentional Nazi salute in Germany, where making such gestures is illegal. The Anti-Defamation League said it was not a Nazi salute, but other Jewish organizations disagreed and condemned the salute. American public opinion was divided on partisan lines as to whether it was a fascist salute. Musk dismissed the accusations of Nazi sympathies, deriding them as "dirty tricks" and a "tired" attack. Neo-Nazi and white supremacist groups celebrated it as a Nazi salute. Multiple European political parties demanded that Musk be banned from entering their countries.\n\n\n\nPage: Acquisition of Twitter by Elon Musk\nSummary: Businessman and U.S. political figure Elon Musk initiated an acquisit

Trace(trace_id=tr-3249ae06a2b85877e6cef16f3f9a5eb9)

In [21]:
from langchain.tools.tavily_search import TavilySearchResults

In [22]:
class LinkedinInputs(BaseModel):
    """Inputs to the web search tool."""

    query: Any= Field(
        description="query to look up in web particularly linkedin data, should be a list"
    )


@mlflow.trace
def linkedin_tool_func(entity_name_1:str):
    linkedin_tool = TavilySearchResults(max_results=3,
                                        include_answer=True,
            include_raw_content=True,
            include_images=True,
            name="Linkedin-tool",
    description="Look up the latest information about the entity_name specified in the web spcifically linkedin page  and return a concise 2 line response highlighting financial news related to the entity",
    args_schema=LinkedinInputs,


    
    
    return_direct=True,
    metadata={"use_case": "entity_analysis", "source": "Linkedin"}
)
    return linkedin_tool.run(entity_name)


linkedin_tool_func(entity_name)



/tmp/ipykernel_4257/3361110606.py:11: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  linkedin_tool = TavilySearchResults(max_results=3,


[{'title': "What happened to Jim Cramer's love affair with Tesla and Elon Musk?",
  'url': 'https://www.teslarati.com/tesla-jim-cramer-love-affair-elon-musk/',
  'content': "“With Musk’s (Tesla stock) purchase, combined with the upward momentum for delivery expectations and robotaxi rollout, we are becoming more bullish.”\n\nImage 45\n\nPublished\n\n4 days ago\non\n\nSeptember 15, 2025\n\nBy\n\nJoey Klender\n\nTesla CEO Elon Musk purchased roughly $1 billion in Tesla shares on Friday, and analysts are now breaking down the move as the stock is headed upward. [...] ------------------------------------------------------------------ [...] Published Time: 2021-09-29T14:59:47+00:00\n\nWhat happened to Jim Cramer's love affair with Tesla and Elon Musk?\n\nImage 1: TESLARATI\n\n   NEWS\n   Tesla\n   SPACEX\n   Elon Musk\n   Shop\n\nConnect with us\n    shares snatched up by ARK after Battery Day: “It’s going to be hard to catch up”",
  'score': 0.586926,
  'raw_content': 'Published Time: 2021

Trace(trace_id=tr-004c23995ca778e7a97ea7b87ea28ec1)